In [1]:
import pandas as pd
from bqplot import ColorScale, ColorAxis, DateScale, LinearScale, Axis, Lines, Figure
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from ipywidgets import Label, VBox, HBox, Layout, Accordion, Dropdown, widgets, interact
from bqplot.market_map import MarketMap
import os
from datetime import datetime
from iexfinance import get_historical_data
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

## RETRIEVE AND CLEAN DATA

In [2]:
etf_data = pd.read_excel('../us_etfs.xlsx').drop_duplicates()
etf_data.columns = [i.lower() for i in etf_data.columns]
etf_list = list(etf_data['ticker'])
def get_iex_data(stock_list, start, end):
    return_list = []
    for i in stock_list:
        df = pd.DataFrame(get_historical_data(i, start, end, output_format='pandas')).interpolate()
        print(i)
        df['ticker'] = i
        return_list.append(df)
    return return_list

In [56]:
# full = get_iex_data(etf_list, start=None, end=None)

In [5]:
def lstm_clean_data(data):
    for i in range(len(data)):
        data[i] = data[i].reset_index().dropna()
        data[i]['date'] = pd.to_datetime(data[i]['date'])
        data[i] = data[i].set_index('date')
        data[i]['Reg_Target'] = data[i]['close'].shift(-1)
    return data

def add_past(etf_list, times):
    for i in range(len(etf_list)):
        for n in times:
            etf_list[i]['{}day return'.format(n)] = -etf_list[i]['close'].diff(periods=n).round(3)
    return etf_list

In [ ]:
clean_full = lstm_clean_data(full)
data = add_past(clean_full, [1, 5, 21, 252])
# inp = add_past(clean_full, [1,2,3,4,5])

## RUN MODELS

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import timeit

In [7]:
def lstm_time_test_split(X, n_past, date):
    X = X.reset_index()
    scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()
    ticker = X['ticker'].iloc[0]
    x_train = X[X['date'] < date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])
    scaler.fit(x_train)
    x_test = X[X['date'] >= date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])[:-1]
    x_train = scaler.transform(x_train)
    x_train = np.reshape(x_train,(x_train.shape[0], n_past, x_train.shape[1]))
    x_test = scaler.transform(x_test)
    x_test = np.reshape(x_test,(x_test.shape[0], n_past, x_test.shape[1]))
    y_train = np.array(X[X['date'] < date]['Reg_Target'].drop(columns='date')).ravel().astype('float').reshape(-1,1)
    y_scaler.fit(y_train)
    y_train = y_scaler.transform(y_train)
    y_test = np.array(X[X['date'] >= date]['Reg_Target'].drop(columns='date')).ravel().astype('float')[:-1].reshape(-1,1)
    y_test = y_scaler.transform(y_test)
    x_holdout = X[X['date'] >= date].drop(columns=['date', 'Reg_Target', 'ticker', '1day return', '5day return', '21day return', '252day return'])[-1:]
    x_holdout = scaler.transform(x_holdout)
    x_holdout = np.reshape(x_holdout,(x_holdout.shape[0], n_past, x_holdout.shape[1]))
#     y_test = scaler.transform(y_test)
    return ticker, x_train, x_test, x_holdout, y_train, y_test, scaler, y_scaler

def build_step_model(x_train, y_train, epoc):
    model = Sequential()
    model.add(LSTM(50, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('relu'))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(x_train, y_train, epochs=epoc, batch_size=64, validation_split=.07, verbose=2,shuffle=False)
    return model, history

def yield_preds(model, scaler, x_test, x_holdout, y_test):
    yhat = model.predict(x_test)
    preds = scaler.inverse_transform(yhat)
    true = scaler.inverse_transform(y_test)
    today = model.predict(x_holdout)
    today_pred = scaler.inverse_transform(today)
    return preds, today_pred

def run_all_lstms(data, split, epoc):
    out = pd.DataFrame()
    tomorrow = pd.DataFrame()
    start = timeit.default_timer()
    for i in range(len(data)):
        ticker, x_train, x_test, x_holdout, y_train, y_test, scaler, y_scaler = lstm_time_test_split(data[i], 1, split)
        print('Model #: {}'.format(i))
        model, history = build_step_model(x_train, y_train, epoc)
        preds, future = yield_preds(model, y_scaler, x_test, x_holdout, y_test)
        out[ticker] = preds.flatten()
        tomorrow[ticker] = future.flatten()
    out = out.set_index(data[0][-len(out):].index)
    stop = timeit.default_timer()
    print('Time: ', stop - start)
    return out, tomorrow

## MAKE DASHBOARD INPUTS

In [8]:
def make_yesterday_mapframe(ref):
    yesterday_inputs = pd.DataFrame([ref[i].reset_index().iloc[-1] for i in range(len(ref))])
    mapframe = pd.merge(etf_data, yesterday_inputs, on='ticker').drop(columns = 'Reg_Target')
    return mapframe

In [9]:
mapframe_y = make_yesterday_mapframe(data)
mapframe_y.columns = ['ticker','Fund Name','issuer','Expense Ratio','Total Assets','spread (%)','Segment','date','Open','High','Low','Previous Close',
                    'volume','1 Day return', '1 Week return', '1 Month return','1 Year return']

In [10]:
def make_daily_mapframe(ref, value):
    mapframe = pd.DataFrame()
    for i in range(len(ref)):
        ticker = ref[i]['ticker'].iloc[0]
        mapframe[ticker] = ref[i][value]
#     mapframe = mapframe.set_index(ref[0]['date']) 
    return mapframe

In [11]:
mapframe_daily_close = make_daily_mapframe(data, 'close')

In [17]:
mapframe_preds = pd.read_csv('important_backup.csv').set_index('date')

In [54]:
def explore(ticker):
    df = [i for i in full if ticker in list(i['ticker'])][0]
    trace = go.Candlestick(x=df.reset_index()['date'],
                           open=df['open'],
                           high=df['high'],
                           low=df['low'],
                           close=df['close'])
    layout = go.Layout(title='{} Price (USD)'.format(ticker),xaxis = dict(
            rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])),
            rangeslider = dict(
                visible = False)))
    inp = [trace]
    fig = go.Figure(data=inp,layout=layout)
    iplot(fig)
interact(explore, ticker=etf_list)

interactive(children=(Dropdown(description='ticker', options=('SPY', 'IVV', 'VTI', 'VOO', 'QQQ', 'VEA', 'EFA', 'IEFA', 'VWO', 'AGG', 'IJH', 'IWM', 'IEMG', 'IJR', 'IWF', 'VTV', 'VUG', 'IWD', 'BND', 'LQD', 'XLF', 'VNQ', 'EEM', 'VIG', 'GLD', 'BSV', 'VO', 'VB', 'TIP', 'VEU', 'IVW', 'XLK', 'VGT', 'DIA', 'VYM', 'MDY', 'VCSH', 'IWB', 'XLV', 'VCIT', 'XLE', 'IWR', 'DVY', 'USMV', 'VGK', 'EWJ', 'SCHF', 'PFF', 'XLY', 'SDY', 'RSP', 'ITOT', 'IVE', 'SCHX', 'HYG', 'SHV', 'EMB', 'VBR', 'SHY', 'VV', 'SCHB', 'XLI', 'BIV', 'VT', 'MBB', 'BNDX', 'VXUS', 'IWO', 'IWS', 'FLOT', 'IXUS', 'MINT', 'MTUM', 'IGSB', 'SCZ', 'IAU', 'IWN', 'IBB', 'TLT', 'MUB', 'AMLP', 'JNK', 'FDN', 'VOE', 'IWP', 'EZU', 'XLP', 'IWV', 'VBK', 'IEF', 'EFAV', 'ACWI', 'VHT', 'SCHA', 'VFH', 'SCHD', 'IJK', 'GDX', 'SPLV', 'XLU', 'BKLN', 'IEI', 'IJT', 'VMBS', 'SCHG', 'IJS', 'VXF', 'QUAL', 'GOVT', 'IJJ', 'ITA', 'VOT', 'DBEF', 'HDV', 'EFV', 'VNQI', 'EWZ', 'GUNR', 'IGIB', 'SCHP', 'DXJ', 'VSS', 'SCHM', 'VTIP', 'IUSG', 'XBI', 'PRF', 'SCHZ', 'PGX', 'HEDJ', 'FXI', 'KRE', 'OEF', 'SCHE', 'EEMV', 'XLB', 'FNDX', 'CWB', 'SLV', 'SCHV', 'INDA', 'SCHH', 'IUSV', 'SPSB', 'NEAR', 'IDV', 'IYR', 'DGRO', 'FVD', 'IYW', 'VDE', 'EMLC', 'GDXJ', 'MGK', 'FNDF', 'VPL', 'VDC', 'TQQQ', 'GSLC', 'EWT', 'FLRN', 'VIS', 'PCY', 'AAXJ', 'EWY', 'NOBL', 'SPIB', 'VLUE', 'SPYG', 'SJNK', 'SHM', 'EFG', 'XOP', 'VTEB', 'FPE', 'BIL', 'FNDA', 'DON', 'KBE', 'SRLN', 'HEFA', 'MCHI', 'SPDW', 'ACWV', 'VCR', 'EWG', 'FEZ', 'VGSH', 'AMJ', 'IHI', 'FBT', 'TOTL', 'SHYG', 'ACWX', 'SPAB', 'RWX', 'EWC', 'SCHR', 'SCHO', 'FV', 'XLRE', 'VPU', 'VAW', 'TFI', 'HYD', 'VGIT', 'SSO', 'FTSM', 'SPTM', 'IGF', 'RPG', 'PDBC', 'IEUR', 'SPHD', 'EWH', 'QTEC', 'DBC', 'IXN', 'VCLT', 'RWR', 'ICF', 'XT', 'FLGE', 'SLYG', 'DGRW', 'JPST', 'VOOG', 'IEV', 'IYH', 'FTEC', 'EPP', 'IUSB', 'FXL', 'RWO', 'SCHC', 'EMLP', 'SDOG', 'FNDE', 'VONG', 'DES', 'ISTB', 'BLV', 'BOTZ', 'MGV', 'VRP', 'USIG', 'QLD', 'IGV', 'PRFZ', 'ROBO', 'SKYY', 'SLYV', 'STIP', 'ONEQ', 'DLN', 'XLC', 'IOO', 'FNDC', 'BBJP', 'EWU', 'BOND', 'GVI', 'HYLB', 'DEM', 'GBIL', 'IYF', 'QDF', 'RYT', 'DLS', 'FTSL', 'HACK', 'PDP', 'IXJ', 'FBGX', 'FAS', 'USO', 'TDTT', 'FXR', 'SPLG', 'IGM', 'HEZU', 'TBT', 'IXC', 'GEM', 'HYS', 'IYG', 'VTWO', 'LMBS', 'ITM', 'FIYY', 'XAR', 'PZA', 'RSX', 'VONV', 'SOXX', 'SPYV', 'JPIN', 'SUB', 'USMC', 'SPSM', 'FEX', 'MGC', 'MOAT', 'CWI', 'GNR', 'MLPI', 'FIHD', 'FNCL', 'PGF', 'KWEB', 'BBCA', 'SPEM', 'FHLC', 'GSY', 'XMLV', 'FDL', 'EPI', 'UPRO', 'ARKK', 'GSG', 'XSLV', 'DGS', 'PXF', 'SLQD', 'PSCH', 'BBEU', 'SPHQ', 'TILT', 'MDYG', 'PWV', 'IPE', 'PTLC', 'OIH', 'FXH', 'LRGF', 'PKW', 'EUFN', 'BSCK', 'EWA', 'SH', 'DSI', 'FXO', 'REET', 'HYLS', 'IYY', 'IWY', 'ILF', 'FDT', 'VOX', 'REM', 'SMH', 'FPX', 'EWW', 'QAI', 'EZM', 'BSCJ', 'FTCS', 'FTA', 'GSIE', 'SLY', 'HEWJ', 'BSJJ', 'ANGL', 'CFO', 'FTXO', 'AOR', 'DBEU', 'SPMD', 'DFJ', 'SPTL', 'IWC', 'CMF', 'PXH', 'VIOO', 'NUGT', 'BWX', 'JKE', 'VXX', 'IYE', 'IYJ', 'TLTD', 'PPA', 'GXC', 'INTF', 'DBJP', 'EWL', 'VWOB', 'AIA', 'RWL', 'BSCL', 'JKD', 'RPV', 'FTC', 'VYMI', 'XNTK', 'IYC', 'PHB', 'IVOO', 'VONE', 'EWP', 'SDIV', 'IGE', 'FNX', 'DHS', 'DFE', 'IHF', 'MDYV', 'TDIV', 'BSJK', 'KBWB', 'SPXL', 'ITB', 'TDTF', 'RDIV', 'VIGI', 'AOM', 'MLPA', 'AOA', 'ASHR', 'IDEV', 'DWM', 'IPAC', 'BAB', 'IYT', 'EWQ', 'DJP', 'INDY', 'VOOV', 'UYG', 'FEP', 'DWX', 'GWX', 'IAGG', 'IGOV', 'IQDF', 'PHO', 'CSM', 'LIT', 'IAT', 'IVOG', 'CIBR', 'BSCM', 'MOO', 'CDC', 'DTN', 'STPZ', 'SDS', 'RODM', 'XHE', 'IBDM', 'XRT', 'CORP', 'NFRA', 'SUSA', 'TECL', 'KIE', 'IFV', 'FYX', 'PID', 'SGOL', 'PEY', 'USRT', 'EES', 'IBDL', 'XME', 'COMT', 'FVC', 'TNA', 'JKG', 'XLG', 'RYH', 'FFEU', 'GDVD', 'IVOV', 'PCEF', 'XHB', 'SPYD', 'NANR', 'FDIS', 'BSCI', 'RDVY', 'MDIV', 'VGLT', 'IBDK', 'PWB', 'PSK', 'JNUG', 'MJ', 'ARKW', 'IYM', 'CFA', 'PTMC', 'PNQI', 'VUSE', 'SOXL', 'RFG', 'ONEO', 'EDV', 'DTD', 'SQQQ', 'SNLN', 'AMZA', 'VIDI', 'VNLA', 'BBAX', 'MNA', 'FREL', 'LVHD', 'USCI', 'IDU', 'TLH', 'EWM', 'EWS', 'FENY', 'RFDI', 'IYZ', 'BSJI', 'PJP', 'CGW', 'QQEW', 'ITE', 'JPUS', 'FFTY', 'IGLB', 'SPTS', 'FIEE', 'TLTE', 'KXI', 'IBUY', 'PFX

<function __main__.explore>

In [57]:
col = ColorScale(scheme='Portland')
segments = mapframe_y['Segment'].values
market_map1 = MarketMap(names=etf_list,      
                       # basic data which needs to set for each map
                       ref_data=mapframe_y, groups=segments, 
                       cols=50, row_groups=10,
                       # Data frame which can be used for different properties of the map
                       # Axis and scale for color data
                       tooltip_fields=['Fund Name', 'Segment', 'Expense Ratio', 'Total Assets', 'Previous Close', '1 Day return', '1 Week return', '1 Month return', '1 Year return'],
                       scales={'color': col},
                       layout={'min_width': '1400px', 'min_height': '1000px'})
# market_map1.color = ['1day_change']
market_map1.font_style = {'font-size': '8px', 'fill':'black'}
market_map1.title = 'ETF Market Map'

selector = Dropdown(options=['Segment', '1 Day return', '1 Week return', '1 Month return', '1 Year return'], value='Segment', description='Color By:', disabled=False,)

sc_x = DateScale()
sc_y = LinearScale()
ax_x = Axis(scale=sc_x, grid_lines='dashed', label='Date')
ax_y = Axis(scale=sc_y, orientation='vertical', grid_lines='dashed', label='Price (USD)')
line = Lines(x= mapframe_daily_close['06-2018':].index.values, y=[], scales={'x': sc_x, 'y': sc_y}, colors=['black'])
linep = Lines(x= mapframe_preds.index.values, y=[], scales={'x': sc_x, 'y': sc_y}, colors=['orange'])

ts_output = Figure(marks=[line, linep], axes=[ax_x, ax_y], layout=Layout(width='88%', height='600px'))
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        market_map1.color = mapframe_y[change['new']]
        market_map1.axes = [ColorAxis(scale=col, label=change['new'], visible=True)]
        
def on_box_change(change):
    line.y = mapframe_daily_close[change['new'][-1]]['06-2018':]
    linep.y = mapframe_preds[change['new'][-1]]
    ts_output.title = '{} Price 2018'.format(change['new'][-1])
    
selector.observe(on_change)
market_map1.observe(on_box_change, 'selected')
first_page = VBox([selector, market_map1, ts_output])
first_page

VBox(children=(Dropdown(description='Color By:', options=('Segment', '1 Day return', '1 Week return', '1 Month return', '1 Year return'), value='Segment'), MarketMap(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], cols=50, font_style={'font-size': '8px', 'fill': 'black'}, groups=array(['Equity: U.S. - Large Cap', 'Equity: U.S. - Large Cap',
       'Equity: U.S. - Total Market', ...,
       'Equity: Germany - Total Market',
       'Equity: South Korea - Total Market',
       'Equity: U.K. - Total Market'], dtype=object), layout=Layout(min_height='1000px', min_width='1400px'), map_margin={'top': 50, 'right': 50, 'left': 50, 'bottom': 50}, names=array(['SPY', 'IVV', 'VTI', ..., 'DBGR', 'DBKO', 'DBUK'], dtype='<U4'), ref_data=     ticker                                          Fund Name  \
0       SPY                             SPDR S&P 500 ETF Trust   
1       IVV                           iShares Core S&P 500 ETF   
2       VTI                    Vanguard Total Stock Market ETF   
3       VOO                               Vanguard S&P 500 ETF   
4       QQQ                                  Invesco QQQ Trust   
5       VEA                Vanguard FTSE Developed Markets ETF   
6       EFA                              iShares MSCI EAFE ETF   
7      IEFA                         iShares Core MSCI EAFE ETF   
8       VWO                 Vanguard FTSE Emerging Markets ETF   
9       AGG               iShares Core U.S. Aggregate Bond ETF   
10      IJH                       iShares Core S&P Mid-Cap ETF   
11      IWM                           iShares Russell 2000 ETF   
12     IEMG             iShares Core MSCI Emerging Markets ETF   
13      IJR                     iShares Core S&P Small Cap ETF   
14      IWF                    iShares Russell 1000 Growth ETF   
15      VTV                                 Vanguard Value ETF   
16      VUG                                Vanguard Growth ETF   
17      IWD                     iShares Russell 1000 Value ETF   
18      BND                     Vanguard Total Bond Market ETF   
19      LQD  iShares iBoxx USD Investment Grade Corporate B...   
20      XLF                  Financial Select Sector SPDR Fund   
21      VNQ                           Vanguard Real Estate ETF   
22      EEM                  iShares MSCI Emerging Markets ETF   
23      VIG                 Vanguard Dividend Appreciation ETF   
24      GLD                                    SPDR Gold Trust   
25      BSV                       Vanguard Short-Term Bond ETF   
26       VO                               Vanguard Mid-Cap ETF   
27       VB                             Vanguard Small-Cap ETF   
28      TIP                              iShares TIPS Bond ETF   
29      VEU                  Vanguard FTSE All-World ex-US ETF   
...     ...                                                ...   
1616   SFIG  WisdomTree Fundamental U.S. Short-Term Corpora...   
1617   SFHY  WisdomTree Fundamental U.S. Short-Term High Yi...   
1618   HGSD    WisdomTree Global Hedged SmallCap Dividend Fund   
1619    GSD           WisdomTree Global SmallCap Dividend Fund   
1620   WCHN               WisdomTree ICBCCS S&P China 500 Fund   
1621   DWMF          WisdomTree International Multifactor Fund   
1622   DXJF            WisdomTree Japan Hedged Financials Fund   
1623   JHDG  WisdomTree Japan Hedged Quality Dividend Growt...   
1624   GULF               WisdomTree Middle East Dividend Fund   
1625   AGND  WisdomTree Negative Duration U.S. Aggregate Bo...   
1626    QSY     WisdomTree U.S. Quality Shareholder Yield Fund   
1627   SHAG  WisdomTree Yield Enhanced U.S. Short-Term Aggr...   
1628   EQLT                       Workplace Equality Portfolio   
1629   IFIX  Xtrackers Barclays International Corporate Bon...   
1630   IGVT  Xtrackers Barclays International Treasury Bond...   
1631   EMIH  Xtrackers Emerging Markets Bond Interest Rate ...   
1632   EURZ                  